In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

In [2]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'SPO'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=20)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 1

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [3]:
clear_buyers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",138674,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [4]:
query = """
SELECT 
            distinct bca.customer_id,
            --fa.latitude,
            --fa.longitude,
            s.site_identifier_value AS region_code,
            --dm.source_id AS microzone,
            wc.short_name AS warehouse

FROM        postgres_broadleaf_federate."broadleaf.fb_address" as fa
INNER JOIN  postgres_broadleaf_federate."broadleaf.blc_customer_address" bca ON bca.address_id = fa.address_id
INNER JOIN  postgres_broadleaf_federate."broadleaf.blc_address" adrs ON bca.address_id = adrs.address_id
INNER JOIN  postgres_broadleaf_federate."broadleaf.blc_customer" bc on bc.customer_id=bca.customer_id
INNER JOIN  postgres_broadleaf_federate."broadleaf.blc_site" s ON s.site_id = bc.site_disc
INNER JOIN  dpr_shared.dim_site ds ON ds.identifier_value = s.site_identifier_value
INNER JOIN  dpr_shared.dim_microzone dm ON dm.site_id = ds.site_id
LEFT JOIN   postgres_main_federate."ops_wh_planning.microzone_configuration" mc ON mc.microzone = dm.source_id
LEFT JOIN   postgres_main_federate."ops_wh_planning.warehouse_configuration" wc on mc.warehouse_configuration_id = wc.id

WHERE   
    ST_Contains(dm.boundaries, ST_Point(fa.latitude, fa.longitude))
    AND wc.short_name IS NOT NULL
    AND dm.group_owner_id = 1
    AND s.site_identifier_value = '{city}'
    AND bca.customer_id IN {clients}
""".format(clients=tuple(clear_buyers.loc[clear_buyers.registered_city == city_code].customer_id.unique()), city = city_code)

df_clean_customers = read_connection_data_warehouse.runQuery(query)

In [5]:
df_clean_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66402 entries, 0 to 66401
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  66402 non-null  int64 
 1   region_code  66402 non-null  object
 2   warehouse    66402 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.5+ MB


In [6]:
query = """
WITH
info AS (
SELECT DISTINCT
    s.site_identifier_value as region_code,
    --ffg.close_date,
    bo.customer_id,
    bs.addl_product_id as padre_sku_id,
    --bs2.name AS card_name,
    --bcat2.name AS category,
    bcat.name AS subcat,
    COUNT(DISTINCT EXTRACT(WEEK FROM bo.submit_date)) AS count_weeks,
    --baid.adjustment_reason,
    SUM ( (boi.quantity * foi.step_unit * boi.sale_price)/{tipo_cambio}) AS gmv_usd
    --SUM ( COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit)/{tipo_cambio}, 0) ) AS discount_applied,
    --SUM ( boi.quantity* foi.step_unit ) AS cant

FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs2        ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"              bcat2      ON bcx.category_id = bcat2.category_id
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

WHERE 
   DATE(bo.submit_date) >= DATE_TRUNC('week', date(getdate()) - interval '10 week') --current_date - interval '10 week' 
   AND DATE(bo.submit_date) <=  current_date
   AND bo.order_status='SUBMITTED'
   AND fo.fb_order_status_id IN (1,6,7,8) 
   AND s.site_identifier_value = '{ciudad}'
   AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
   AND bop.archived = 'N'
   AND (fot.name IS NULL OR fot.name <> 'REFUND')
   -- FIX SUPER DESCUENTOS
   AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
   AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
   AND bo.customer_id IN {clientes}
GROUP BY 1,2,3,4
)

SELECT 
*
FROM info
--WHERE count_weeks > 5
""".format(tipo_cambio=t_cambio, ciudad=city_code, clientes=tuple(df_clean_customers.customer_id.unique()))

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd"]] = data_ventas_aux[["gmv_usd"]].astype(float)
#data_ventas["submit_date"] = pd.to_datetime(data_ventas["submit_date"])

In [7]:
data_ventas_aux

,region_code,customer_id,padre_sku_id,subcat,count_weeks,gmv_usd
0,SPO,66552038,-202908,Tubérculos,6,143.03155
1,SPO,69001052,61584,Verduras,10,33.89251
2,SPO,116098419,60989,Verduras,2,2.92420
3,SPO,30503944,-203011,Tubérculos,3,37.74104
4,SPO,134542771,77816,Frutas,2,5.00842
...,...,...,...,...,...,...
578217,SPO,156164386,123459,Arroz,1,10.56000
578218,SPO,81860306,194591,Outros Limpadores,1,1.49684
578219,SPO,317932918,-202941,Legumes,1,12.37894
578220,SPO,55822739,130577,Desengordurantes,1,5.37262


In [8]:
df_ventas_wh = pd.merge(data_ventas_aux, df_clean_customers.loc[df_clean_customers.warehouse.isin(['STA','GRU','VLP'])],  
                        how='inner', left_on=['region_code','customer_id'], right_on = ['region_code','customer_id'])

In [9]:
df_ventas_wh

,region_code,customer_id,padre_sku_id,subcat,count_weeks,gmv_usd,warehouse
0,SPO,66552038,-202908,Tubérculos,6,143.03155,STA
1,SPO,66552038,275011,Batatas Pré-Fritas,1,23.11578,STA
2,SPO,66552038,-202950,Legumes,2,32.24840,STA
3,SPO,66552038,339061,Refrigerantes e Energéticos,1,30.99789,STA
4,SPO,66552038,60544,Legumes,9,270.78304,STA
...,...,...,...,...,...,...,...
615302,SPO,341369085,280421,Cervejas,1,39.31578,GRU
615303,SPO,335074827,293209,"Azeites, Óleos e Vinagres",1,52.80000,VLP
615304,SPO,132142251,60544,Legumes,1,6.43788,STA
615305,SPO,42361390,443309,Leite,1,24.65684,GRU


In [10]:
df_ventas_wh.loc[df_ventas_wh.subcat == 'Arroz'].count_weeks.value_counts()

1     7055
2     2889
3     1650
4     1140
5      738
6      549
7      267
8      200
9      167
10     154
11      39
Name: count_weeks, dtype: int64

In [11]:
df_ventas_wh.loc[df_ventas_wh.padre_sku_id.isin([231959,205461])].count_weeks.value_counts()

1     2573
2     1314
3      846
4      581
5      351
6      234
7       53
8       21
9       19
10      13
11       5
Name: count_weeks, dtype: int64

In [12]:
df_ventas_wh.subcat.unique()

array(['Tubérculos', 'Batatas Pré-Fritas', 'Legumes',
       'Refrigerantes e Energéticos', 'Leite', 'Verduras', 'Queijos',
       'Massas e Molhos', 'Cervejas', 'Café, Chocolates e Infusões',
       'Frutas', 'Bomboniere', 'Feijão', 'Ovos',
       'Azeites, Óleos e Vinagres', 'Temperos',
       'Desinfetantes e Sanitizantes', 'Sucos e Chás',
       'Outros Limpadores', 'Polpas e Açaí', 'Frango', 'Utensílios',
       'Manteigas e Margarinas', 'Águas e Isotônicos', 'Condimentos',
       'Arroz', 'Embutidos', 'Detergente, Sabão e Lava-Louças',
       'Farinhas e Misturas', 'Utensílios de Limpeza', 'Papéis',
       'Açúcar, Adoçantes e Doces', 'Laticínios',
       'Copos, Recipientes e Pratos', 'Alimentos Congelados',
       'Conservas e Enlatados', 'Grãos', 'Doces e Sobremesas',
       'Desengordurantes', 'Sacos', 'Carne Vermelha', 'Suínos',
       'Peixes e Frutos do Mar', 'Embalagens', 'Destilados',
       'Drinks Prontos'], dtype=object)

In [16]:
f = df_ventas_wh.loc[(df_ventas_wh.count_weeks>2) & (df_ventas_wh.subcat.isin(['Arroz','Feijão','Queijos','Café, Chocolates e Infusões','Frango']))].groupby(by=["subcat","warehouse"], as_index=False).agg({'customer_id':[lambda x: x.nunique(),lambda x: list(x.unique())]}).droplevel(0, axis="columns")
f.columns = ['subcat', 'wh', 'customers', 'list_customers']
f

,subcat,wh,customers,list_customers
0,Arroz,GRU,859,"[75904690, 36527659, 85874473, 264102212, 1237..."
1,Arroz,STA,1461,"[69001052, 22695995, 64601928, 40193160, 86744..."
2,Arroz,VLP,1496,"[30503944, 264051436, 22695995, 160290004, 491..."
3,"Café, Chocolates e Infusões",GRU,286,"[89057121, 74310816, 284235984, 123765760, 105..."
4,"Café, Chocolates e Infusões",STA,475,"[22695995, 239569558, 40517173, 42255520, 4799..."
5,"Café, Chocolates e Infusões",VLP,399,"[30503944, 22695995, 56493600, 49110745, 18593..."
6,Feijão,GRU,717,"[75904690, 36527659, 264102212, 123765760, 684..."
7,Feijão,STA,1276,"[66552038, 69001052, 22695995, 64601928, 86744..."
8,Feijão,VLP,1210,"[30503944, 22695995, 49110745, 46652033, 19935..."
9,Frango,GRU,542,"[36527659, 85874473, 298061435, 68411026, 3500..."


In [17]:
def create_groups(subcat, index_wh):
    print(f.loc[index_wh].wh)    
    s = f.loc[(f.subcat == subcat) & (f.wh == f.loc[index_wh].wh)].list_customers.to_numpy()[0]
    #ACA D SON LOS QUE ENTRAN AL PILOTO
    d = df_ventas_wh[(df_ventas_wh.customer_id.isin(s)) & (df_ventas_wh.subcat == subcat)].groupby("customer_id").gmv_usd.mean().reset_index()
    
    customer_control, customer_test = train_test_split(s, test_size = 0.50)
    
    alpha=0.01
    aux=0
    
    while abs(1-(d[d.customer_id.isin(customer_control)]['gmv_usd'].mean()/d[d.customer_id.isin(customer_test)]['gmv_usd'].mean())) > alpha:
        customer_control, customer_test = train_test_split(s, test_size = 0.50)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
            
    #all_users = clear_buyers[(~clear_buyers.customer_id.isin(customer_test)) & (clear_buyers.registered_city == city_code)].customer_id.unique().tolist()
    customer_control_L = [int(customer) for customer in customer_control]
    customer_test_L = [int(customer) for customer in customer_test]
    
    segt_control_L = {'name':"HOOK_Localizacion_"+f.loc[index_wh].wh+"_"+todays_date+"_"+city_code+"_"+subcat+"_CONTROL_LL2", 'customersIds': list(customer_control_L)}
    segt_test_L = {'name':"HOOK_Localizacion_"+f.loc[index_wh].wh+"_"+todays_date+"_"+city_code+"_"+subcat+"_TEST_LL2", 'customersIds': list(customer_test_L)}
    
    print("L1 INFO SUBCAT:",subcat, "// TEST:",len(customer_test_L), "// CONTROL:",len(customer_control_L))

#ALL --------------------------------------------------------------------------
    all_users = df_clean_customers[(~df_clean_customers.customer_id.isin(segt_test_L)) & (clear_buyers.registered_city == city_code)].customer_id.unique().tolist()
    
    print('ALL: ', len(all_users))
    
    return customer_control_L, customer_test_L, segt_test_L, segt_control_L

In [18]:
global_segment_list = []
r= {}

for wh_i, subcat in enumerate(f.subcat):
    control_M2, test_M2, segment_t, segment_c = create_groups(subcat, wh_i)
    #create segments format
    global_segment_list.append(segment_t)
    global_segment_list.append(segment_c)

GRU
L1 INFO SUBCAT: Arroz // TEST: 430 // CONTROL: 429


/home/jupyter-felipe.guevara/.local/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


ALL:  14606
STA
L1 INFO SUBCAT: Arroz // TEST: 731 // CONTROL: 730
ALL:  14606
VLP
L1 INFO SUBCAT: Arroz // TEST: 748 // CONTROL: 748
ALL:  14606
GRU
L1 INFO SUBCAT: Café, Chocolates e Infusões // TEST: 143 // CONTROL: 143
ALL:  14606
STA
L1 INFO SUBCAT: Café, Chocolates e Infusões // TEST: 238 // CONTROL: 237
ALL:  14606
VLP
L1 INFO SUBCAT: Café, Chocolates e Infusões // TEST: 200 // CONTROL: 199
ALL:  14606
GRU
L1 INFO SUBCAT: Feijão // TEST: 359 // CONTROL: 358
ALL:  14606
STA
L1 INFO SUBCAT: Feijão // TEST: 638 // CONTROL: 638
ALL:  14606
VLP
L1 INFO SUBCAT: Feijão // TEST: 605 // CONTROL: 605
ALL:  14606
GRU
L1 INFO SUBCAT: Frango // TEST: 271 // CONTROL: 271
ALL:  14606
STA
L1 INFO SUBCAT: Frango // TEST: 439 // CONTROL: 439
ALL:  14606
VLP
L1 INFO SUBCAT: Frango // TEST: 401 // CONTROL: 401
ALL:  14606
GRU
L1 INFO SUBCAT: Queijos // TEST: 332 // CONTROL: 331
ALL:  14606
STA
L1 INFO SUBCAT: Queijos // TEST: 474 // CONTROL: 473
ALL:  14606
VLP
L1 INFO SUBCAT: Queijos // TEST: 474 

# cst sgt

In [19]:
def crear_seg(global_value):
  import requests
  import json

  url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

  payload = json.dumps(global_value)


  headers = {
    'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  return response.text

In [20]:
sgt = []
i=0
for global_seg in global_segment_list:
    rep_text = crear_seg([global_seg])
    sgt.append(rep_text)
    #sgt.append((rep_text[32:].strip('][')[:6], rep_text[32:].strip('][')[38:]))
    #i+=1
    #if i == 3:
    #    break
    time.sleep(20)
sgt

['Customer segments were created: [157283-HOOK_Localizacion_GRU_2023-05-02_SPO_Arroz_TEST_LL2]',
 'Customer segments were created: [157652-HOOK_Localizacion_GRU_2023-05-02_SPO_Arroz_CONTROL_LL2]',
 'Customer segments were created: [157412-HOOK_Localizacion_STA_2023-05-02_SPO_Arroz_TEST_LL2]',
 'Customer segments were created: [155133-HOOK_Localizacion_STA_2023-05-02_SPO_Arroz_CONTROL_LL2]',
 'Customer segments were created: [155134-HOOK_Localizacion_VLP_2023-05-02_SPO_Arroz_TEST_LL2]',
 'Customer segments were created: [155135-HOOK_Localizacion_VLP_2023-05-02_SPO_Arroz_CONTROL_LL2]',
 'Customer segments were created: [157565-HOOK_Localizacion_GRU_2023-05-02_SPO_Café, Chocolates e Infusões_TEST_LL2]',
 'Customer segments were created: [157472-HOOK_Localizacion_GRU_2023-05-02_SPO_Café, Chocolates e Infusões_CONTROL_LL2]',
 'Customer segments were created: [157606-HOOK_Localizacion_STA_2023-05-02_SPO_Café, Chocolates e Infusões_TEST_LL2]',
 'Customer segments were created: [157607-HOOK_Lo

In [21]:
for x in sgt:
    print(x[32:].strip('][')[:6]+"\t"+x[32:].strip('][')[25:])

157283	GRU_2023-05-02_SPO_Arroz_TEST_LL2
157652	GRU_2023-05-02_SPO_Arroz_CONTROL_LL2
157412	STA_2023-05-02_SPO_Arroz_TEST_LL2
155133	STA_2023-05-02_SPO_Arroz_CONTROL_LL2
155134	VLP_2023-05-02_SPO_Arroz_TEST_LL2
155135	VLP_2023-05-02_SPO_Arroz_CONTROL_LL2
157565	GRU_2023-05-02_SPO_Café, Chocolates e Infusões_TEST_LL2
157472	GRU_2023-05-02_SPO_Café, Chocolates e Infusões_CONTROL_LL2
157606	STA_2023-05-02_SPO_Café, Chocolates e Infusões_TEST_LL2
157607	STA_2023-05-02_SPO_Café, Chocolates e Infusões_CONTROL_LL2
157332	VLP_2023-05-02_SPO_Café, Chocolates e Infusões_TEST_LL2
155136	VLP_2023-05-02_SPO_Café, Chocolates e Infusões_CONTROL_LL2
157566	GRU_2023-05-02_SPO_Feijão_TEST_LL2
157413	GRU_2023-05-02_SPO_Feijão_CONTROL_LL2
157608	STA_2023-05-02_SPO_Feijão_TEST_LL2
157567	STA_2023-05-02_SPO_Feijão_CONTROL_LL2
155137	VLP_2023-05-02_SPO_Feijão_TEST_LL2
157414	VLP_2023-05-02_SPO_Feijão_CONTROL_LL2
157473	GRU_2023-05-02_SPO_Frango_TEST_LL2
157568	GRU_2023-05-02_SPO_Frango_CONTROL_LL2
157284	STA